In [ ]:
!pip install scapy

In [ ]:
import re
import socket
from scapy.all import ARP, Ether, srp, conf
conf.verb = 0

def arp_scan(network, timeout=2):
    pkt = Ether(dst="ff:ff:ff:ff:ff:ff")/ARP(pdst=network)
    ans, _ = srp(pkt, timeout=timeout)
    return [{"ip": rcv.psrc, "mac": rcv.hwsrc} for snd, rcv in ans]

def is_valid_network(network):
    return re.match(r"^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}/\d{1,2}$", network) is not None

def get_hostname(ip):
    try:
        return socket.gethostbyaddr(ip)[0]
    except socket.error:
        return "-"

def main():
    target_network = input("Enter the target network address:").strip()

    print("--------------------------------------------")

    if not is_valid_network(target_network):
        print("\n[!] Error: Invalid network format. Please use CIDR notation (e.g., 192.168.1.0/24).")
    else:
        print(f"\n Starting ARP scan on {target_network}...")
        devices = arp_scan(target_network)

        print("\n---- Discovered Devices ---")
        if devices:
            print(f"{'IP Address':<18}{'MAC Address':<20}{'Hostname'}")
            print("-" * 65) 
            
            for d in devices:
                ip = d['ip']
                hostname = get_hostname(ip)
                
                print(f"{ip:<18}{d['mac']:<20}{hostname}") 
        else:
            print("No devices responded.")

        print("---------------------------")

if __name__ == "__main__":
    main()